## Tools Calling , Function Calling

## all we need to do is python function and llm is going to call this python function

In [14]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GEMINI_API_KEY")

from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp")

## Understanding the tool calling

In [15]:
from langchain_core.messages import HumanMessage,ToolMessage   # AIMessage,SystemMessage
from langchain_core.tools import tool

## Define Tool

In [16]:
## datatypes for input, output and description is important
@tool
def bmi_calculator(weight:float, height:float) -> str:
    """
    Calculate the BMI of a person given their weight and height.
    """

    bmi = weight/(height**2)
    if bmi < 18.5:
        return "Underweight"
    elif bmi > 18.5 and bmi < 24.9:
        return "Normal weight"
    elif bmi > 24.9 and bmi < 29.9:
        return "Overweight"
    else:
        return "Obesity"
    

tools = [bmi_calculator]

In [17]:
llm_with_tool = llm.bind_tools(tools)

query = "What is the BMI of a person who is 100kg and 1.75m tall ?"

message = [HumanMessage(content=query)]
ai_message = llm_with_tool.invoke(message)

message

[HumanMessage(content='What is the BMI of a person who is 100kg and 1.75m tall ?', additional_kwargs={}, response_metadata={})]

In [18]:
ai_message

AIMessage(content='', additional_kwargs={'function_call': {'name': 'bmi_calculator', 'arguments': '{"height": 1.75, "weight": 100.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-4a036591-1221-4f91-9a6c-203bf81b59bb-0', tool_calls=[{'name': 'bmi_calculator', 'args': {'height': 1.75, 'weight': 100.0}, 'id': 'bd7e982e-4df5-4651-9310-51db091fb96a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 42, 'output_tokens': 5, 'total_tokens': 47, 'input_token_details': {'cache_read': 0}})

In [19]:
message.append(ai_message)

print(message)

[HumanMessage(content='What is the BMI of a person who is 100kg and 1.75m tall ?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'function_call': {'name': 'bmi_calculator', 'arguments': '{"height": 1.75, "weight": 100.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-4a036591-1221-4f91-9a6c-203bf81b59bb-0', tool_calls=[{'name': 'bmi_calculator', 'args': {'height': 1.75, 'weight': 100.0}, 'id': 'bd7e982e-4df5-4651-9310-51db091fb96a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 42, 'output_tokens': 5, 'total_tokens': 47, 'input_token_details': {'cache_read': 0}})]


In [20]:
for i in ai_message.tool_calls:
    print(i)

{'name': 'bmi_calculator', 'args': {'height': 1.75, 'weight': 100.0}, 'id': 'bd7e982e-4df5-4651-9310-51db091fb96a', 'type': 'tool_call'}


In [21]:
for i in ai_message.tool_calls:
    selected_tools = {"bmi_calculator": bmi_calculator}[i['name'].lower()]
    print(selected_tools)

name='bmi_calculator' description='Calculate the BMI of a person given their weight and height.' args_schema=<class 'langchain_core.utils.pydantic.bmi_calculator'> func=<function bmi_calculator at 0x10d736480>


In [22]:
for i in ai_message.tool_calls:
    selected_tools = {"bmi_calculator": bmi_calculator}[i['name'].lower()]
    print(selected_tools)
    tool_output = selected_tools.invoke(i['args'])
    print(tool_output)
    message.append(ToolMessage(content=tool_output, tool_call_id=i['id']))

print(message)

name='bmi_calculator' description='Calculate the BMI of a person given their weight and height.' args_schema=<class 'langchain_core.utils.pydantic.bmi_calculator'> func=<function bmi_calculator at 0x10d736480>
Obesity
[HumanMessage(content='What is the BMI of a person who is 100kg and 1.75m tall ?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'function_call': {'name': 'bmi_calculator', 'arguments': '{"height": 1.75, "weight": 100.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-4a036591-1221-4f91-9a6c-203bf81b59bb-0', tool_calls=[{'name': 'bmi_calculator', 'args': {'height': 1.75, 'weight': 100.0}, 'id': 'bd7e982e-4df5-4651-9310-51db091fb96a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 42, 'output_tokens': 5, 'total_tokens': 47, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='Obesity', tool_call_id='bd7e982e-4df5-4651-9310

[HumanMessage(content='What is the BMI of a person who is 100kg and 1.75m tall ?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'function_call': {'name': 'bmi_calculator', 'arguments': '{"height": 1.75, "weight": 100.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-4a036591-1221-4f91-9a6c-203bf81b59bb-0', tool_calls=[{'name': 'bmi_calculator', 'args': {'height': 1.75, 'weight': 100.0}, 'id': 'bd7e982e-4df5-4651-9310-51db091fb96a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 42, 'output_tokens': 5, 'total_tokens': 47, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='Obesity', tool_call_id='bd7e982e-4df5-4651-9310-51db091fb96a')]

In [ ]:
response = llm_with_tool.invoke(message)
print(response.content)

ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 * GenerateContentRequest.contents[2].parts[0].function_response.name: Name cannot be empty.
